# Fine Tuning de Modelos de Lenguaje

[![Abrir en Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aldomunaretto/immune_generative_ai/blob/main/notebooks/05_fine_tuning.ipynb)
 
Este notebook guía paso a paso el proceso de ajuste fino (fine-tuning) de un modelo de lenguaje utilizando la librería Unsloth y Hugging Face. En el realizaremos el ajuste fino (fine tuning) de un modelo de lenguaje para tareas de extracción de información a partir de fragmentos HTML.

### Instalación de dependencias en Google Colab

In [ ]:
%%capture
import os, importlib.util
!pip install --upgrade -qqq uv
if importlib.util.find_spec("torch") is None or "COLAB_" in "".join(os.environ.keys()):
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    !uv pip install -qqq \
        "torch>=2.8.0" "triton>=3.4.0" {get_numpy} {get_pil} torchvision bitsandbytes "transformers==4.56.2" \
        "unsloth_zoo[base] @ git+https://github.com/unslothai/unsloth-zoo" \
        "unsloth[base] @ git+https://github.com/unslothai/unsloth" \
        git+https://github.com/triton-lang/triton.git@0add68262ab0a2e33b84524346cb27cbb2787356#subdirectory=python/triton_kernels
elif importlib.util.find_spec("unsloth") is None:
    !uv pip install -qqq unsloth
!uv pip install --upgrade --no-deps transformers==4.56.2 tokenizers trl==0.22.2 unsloth unsloth_zoo

### Instalación de dependencias en entorno local

In [ ]:
!pip install unsloth

### Importación de librerías necesarias

In [ ]:
import os
import json
import unsloth
import torch
from google.colab import files
from unsloth import FastLanguageModel
from trl import SFTTrainer
from datasets import load_dataset, Dataset
from transformers import TrainingArguments

### Verificación de GPU

In [ ]:
print(f"CUDA disponible: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No se detecta ninguna GPU'}")

### Carga del modelo y tokenizador
 
En este paso se descarga y prepara el modelo base (Llama-3.2-3B-Instruct) y su tokenizador usando la función de Unsloth. Se configuran parámetros como la longitud máxima de secuencia y el uso de 4 bits para optimizar memoria y velocidad.

In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct",
    max_seq_length = 2048,
    load_in_4bit = True,
    dtype = None,
)

### Carga del dataset en formato JSON
 
Se carga el dataset que contiene ejemplos de entrada y salida para el ajuste fino. El archivo debe estar en formato JSON y estructurado para tareas de extracción de información.

In [ ]:
dataset = load_dataset("json", data_files="json_extraction_dataset_500.json")

### Definición de la función de formateo del dataset
 
Se crea una función para transformar cada ejemplo del dataset al formato requerido por el modelo. Esto incluye convertir la salida a JSON y estructurar los mensajes en el formato de chat esperado por el modelo.

In [ ]:
def format_example(row):
    # Convertimos el output (diccionario) a string JSON
    answer = json.dumps(row["output"], ensure_ascii=False)

    messages = [
        {"role": "system", "content": "Eres un asistente que extrae información de productos a partir de fragmentos HTML."},
        {"role": "user", "content": row["input"]},
        {"role": "assistant", "content": answer},
    ]

    # Aplicamos el template de chat
    row["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return row

### Aplicación del formateo al dataset
 
Se aplica la función de formateo a todos los ejemplos del dataset para que estén listos para el entrenamiento supervisado.

In [ ]:
dataset = dataset["train"].map(format_example, batched=False)

### Configuración del adaptador LoRA para fine-tuning eficiente
 
Se utiliza la técnica LoRA (Low-Rank Adaptation) para ajustar el modelo de manera eficiente, reduciendo el número de parámetros entrenables y el consumo de memoria. Aquí se configuran los hiperparámetros principales del adaptador.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0.0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
    use_rslora = False,
    loftq_config = None,
)

### Configuración y ejecución del entrenamiento supervisado (SFTTrainer)
 
Se configura el entrenador supervisado (SFTTrainer) con los hiperparámetros de entrenamiento, el modelo, el tokenizador y el dataset ya formateado. Aquí se define el número de épocas, el tamaño de batch, el optimizador y otros parámetros clave para el fine-tuning.

In [ ]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    dataset_text_field="text",
    train_dataset = dataset,
    args = TrainingArguments(
      output_dir = "outputs",
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=3,
      max_steps = 500,
      learning_rate=1e-4,
      fp16=True,
      logging_steps=50,
      optim = "adamw_8bit",
      weight_decay = 0.01,
      lr_scheduler_type = "linear",
      save_steps=500,
      save_total_limit=2,
      report_to="none"
      )
)

trainer.train()

### Evaluación del modelo afinado
 
Después del entrenamiento, se prueba el modelo con un ejemplo de entrada para verificar que ha aprendido a extraer la información correctamente. Se utiliza un pipeline de generación de texto para obtener la respuesta del modelo.

In [ ]:
from transformers import pipeline

gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=300)

prueba_input = "Extrae información del siguiente producto:\n<div class='product'><h2>Test Product</h2><span class='price'>$999</span><span class='category'>electronics</span><span class='brand'>OpenAI</span></div>"

messages = [
    {"role": "system", "content": "Eres un asistente que extrae información de productos a partir de fragmentos HTML."},
    {"role": "user", "content": prueba_input},
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

generated = gen_pipeline(prompt)[0]["generated_text"]
print("\n=== Salida generada ===\n")
print(generated)

### Guardado del modelo afinado en formato GGUF
 
Se guarda el modelo ajustado en formato GGUF, que es eficiente para su despliegue y uso posterior. También se almacena el tokenizador junto con el modelo.

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="fast_quantized")

### Descarga del modelo para uso local
 
Si trabajas en Google Colab o en un entorno remoto, este bloque permite descargar el archivo del modelo afinado a tu equipo local para su uso o respaldo.

In [ ]:
for gguf_file in [f for f in os.listdir() if f.endswith(".gguf")]:
    print(f"Downloading {gguf_file}")
    files.download(gguf_file)